In [ ]:
!pip install -U langchain langchain-community langchain-openai langchain-core beautifulsoup4 pypdf chromadb

In [ ]:
import os
from getpass import getpass
import os
#Enter the Secret Key here!
os.environ["OPENAI_API_KEY"] = ""


In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# ۱. تنظیمات اولیه
#os.environ["OPENAI_API_KEY"] = "your_api_key"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ۲. بارگذاری و چانک‌بندی
loader = PyPDFLoader("/content/drive/MyDrive/Think-And-Grow-Rich_2011-06.pdf")
docs = loader.load()
print(docs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

# ۳. ذخیره در دیتابیس برداری
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

# ۴. طراحی پرامپت هوشمند با قابلیت درک تاریخچه (History)
contextualize_q_system_prompt = (
    "با توجه به تاریخچه چت و آخرین سوال کاربر که ممکن است به سوالات قبلی ارجاع داشته باشد، "
    "یک سوال مستقل بساز که بدون نیاز به تاریخچه قابل فهم باشد."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# ۵. پرامپت پاسخ‌دهی اصلی
qa_system_prompt = (
    "تو یک دستیار هوشمند هستی. از تکه‌های متن استخراج شده زیر برای پاسخ به سوال استفاده کن. "
    "اگر پاسخ در متن نیست، بگو که نمی‌دانم. حداکثر در سه جمله پاسخ بده."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# ۶. ساخت زنجیره با استفاده از LCEL (بدون استفاده از Chains قدیمی)
# ۶. اصلاح شده: جایگزین کردن get_relevant_documents با invoke
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# زنجیره جدید و سازگار با پایتون ۳.۱۲ و لنگ‌چین جدید
rag_chain = (
    RunnablePassthrough.assign(
        context=lambda x: format_docs(retriever.invoke(x["input"])) # اینجا invoke جایگزین شد
    )
    | qa_prompt
    | llm
    | StrOutputParser()
)

# ۷. اجرای چت با حافظه
chat_history = []

print("--- سیستم آماده است ---")
while True:
    user_input = input("سوال شما: ")
    if user_input.lower() in ['exit', 'quit']: break

    # اجرای زنجیره
    response = rag_chain.invoke({"input": user_input, "chat_history": chat_history})

    print(f"پاسخ: {response}")

    # آپدیت حافظه
    chat_history.append(("human", user_input))
    chat_history.append(("ai", response))